In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def toUpperNoSpaces(row,column):
    return str(row[column]).replace(" ","").upper().strip()
def BoxOfficeNumber(row,column):
    return float(str(row[column]).replace("$","").replace(",","").strip())

dfAll = pd.read_csv('netflixList.csv')
dfOM = pd.read_csv('netflixOM.csv')
dfOS = pd.read_csv('netflixOS.csv')

# Adding new columns for operations like: compare between datasets
dfOM['TitleToCompare'] = dfOM.apply(lambda row: toUpperNoSpaces(row,'Title'), axis = 1)
dfOS['TitleToCompare'] = dfOS.apply(lambda row: toUpperNoSpaces(row,'Title'), axis = 1)
dfAll['TitleToCompare'] = dfAll.apply(lambda row: toUpperNoSpaces(row,'Title'), axis = 1)

dfAll['GenreToCompare'] = dfAll.apply(lambda row: toUpperNoSpaces(row,'Genre'), axis = 1)
dfAll['BoxOfficeProfits'] = dfAll.apply(lambda row: BoxOfficeNumber(row,'Boxoffice'), axis = 1)

# add new column: Original, 1: is produced by netflix, 0: is not produced by netflix
df = dfAll.assign(Original = dfAll.TitleToCompare.isin(pd.concat([dfOM.TitleToCompare,dfOS.TitleToCompare])).astype(int))
df['Year'] = pd.to_numeric(df['Release Date'].str[-4:])

#df: is the result dataset after matching

#  ------------ columns ------------------------
#Index(['Title', 'Genre', 'Tags', 'Languages', 'Series or Movie',
#       'Hidden Gem Score', 'Country Availability', 'Runtime', 'Director',
#       'Writer', 'Actors', 'View Rating', 'IMDb Score',
#       'Rotten Tomatoes Score', 'Metacritic Score', 'Awards Received',
#       'Awards Nominated For', 'Boxoffice', 'Release Date',
#       'Netflix Release Date', 'Production House', 'Netflix Link', 'IMDb Link',
#       'Summary', 'IMDb Votes', 'Image', 'Poster', 'TMDb Trailer',
#       'Trailer Site', 'TitleToCompare', 'Original'],
#      dtype='object')

df

In [25]:
#GRAPH 1: TREND OF "--GENRE--" MOVIES/SERIES FROM NETFLIX

dfTrend = pd.DataFrame()
# get unique values from Genre
dfTemp = df['GenreToCompare'].drop_duplicates().str.split(',', expand=True)
dfGenre = dfTemp[0].drop_duplicates()
for col in dfTemp.columns:
    dfGenre = dfGenre.append(dfTemp[col].drop_duplicates(), ignore_index=True)

# drop duplicates and "Nan" value from Genre unique values 
dfGenre = dfGenre.drop_duplicates().dropna().drop(dfGenre.index[10])

filterYear = df['Year'] >= 2010
for genre in dfGenre:
    filterGenre = df['GenreToCompare'].str.contains(pat = genre)
    dfTrend = dfTrend.append(df[['Original','Title','Series or Movie','Year','GenreToCompare','IMDb Score']].where(filterYear & filterGenre, inplace = False).dropna().drop_duplicates().groupby(['Year','Original','Series or Movie']).agg(Films = ('Title', 'count'), Rating = ('IMDb Score', 'mean')).assign(Genre = genre))

dfTrend = dfTrend.reset_index()

#export to csv file
dfTrend.sort_values(['Genre','Year','Original','Series or Movie'], ascending=[True,True,True,True], inplace=True)
dfTrend
#dfTrend.to_csv(r'D:\COMPUTER SCIENCE\2 SEMESTER\information visualization\project\trend_genre.csv', index = False, header=True)


,Year,Original,Series or Movie,Films,Rating,Genre
192,2010.0,0.0,Movie,75,6.472000,ACTION
193,2010.0,0.0,Series,13,7.638462,ACTION
194,2011.0,0.0,Movie,80,6.270000,ACTION
195,2011.0,0.0,Series,13,7.292308,ACTION
196,2012.0,0.0,Movie,96,6.429167,ACTION
...,...,...,...,...,...,...
834,2018.0,0.0,Movie,7,6.642857,WESTERN
835,2018.0,1.0,Movie,1,7.300000,WESTERN
836,2019.0,0.0,Movie,4,5.325000,WESTERN
837,2019.0,1.0,Movie,1,5.200000,WESTERN


In [58]:
# GRAPH 4: BOX OFFICE CONTRIBUTION

dfBoxOffice = pd.DataFrame()

filterYear = df['Year'] >= 2010
for genre in dfGenre:
    filterGenre = df['GenreToCompare'].str.contains(pat = genre)
    dfBoxOffice = dfBoxOffice.append(df[['Original','Genre','Title','GenreToCompare','Series or Movie','IMDb Score','Year','BoxOfficeProfits']].where(filterYear & filterGenre, inplace = False).dropna(how='all').drop_duplicates().assign(MainGenre = genre))
dfBoxOffice.sort_values(['MainGenre','Year','Original','Series or Movie'], ascending=[True,True,True,True], inplace=True)

dfBoxOffice['BoxOfficeProfits'] = dfBoxOffice['BoxOfficeProfits'].fillna(0)
dfBoxOffice['IMDb Score'] = dfBoxOffice['IMDb Score'].fillna(0)

# adding color
dfBoxOffice['color'] = '#ff3351'
dfBoxOffice.loc[dfBoxOffice['BoxOfficeProfits'] > 0, 'color'] = '#001bff'
dfBoxOffice

#export to csv file
#dfBoxOffice[['MainGenre','Year','Original','Series or Movie','IMDb Score','BoxOfficeProfits','color','Title','Genre']].to_csv(r'D:\COMPUTER SCIENCE\2 SEMESTER\information visualization\project\boxOffice.csv', index = False, header=True)

,Original,Genre,Title,GenreToCompare,Series or Movie,IMDb Score,Year,BoxOfficeProfits,MainGenre,color
79,0.0,"Action, Drama, Thriller",Golden Slumber,"ACTION,DRAMA,THRILLER",Movie,7.1,2010.0,0.0,ACTION,#ff3351
109,0.0,"Action, Comedy, Crime, Thriller",Red,"ACTION,COMEDY,CRIME,THRILLER",Movie,7.0,2010.0,90380162.0,ACTION,#001bff
1951,0.0,"Action, Crime, Drama, Thriller",The Outrage,"ACTION,CRIME,DRAMA,THRILLER",Movie,6.8,2010.0,44745.0,ACTION,#001bff
4817,0.0,"Action, Comedy",Dabangg,"ACTION,COMEDY",Movie,6.2,2010.0,1288549.0,ACTION,#001bff
6676,0.0,Action,Wind Blast,ACTION,Movie,5.1,2010.0,0.0,ACTION,#ff3351
...,...,...,...,...,...,...,...,...,...,...
7690,0.0,"Drama, Thriller, Western","Sonora, The Devil’s Highway","DRAMA,THRILLER,WESTERN",Movie,6.3,2019.0,0.0,WESTERN,#ff3351
4921,1.0,Western,The Outsider,WESTERN,Movie,5.2,2019.0,0.0,WESTERN,#ff3351
177,0.0,"Action, Adventure, Drama, Western",News of the World,"ACTION,ADVENTURE,DRAMA,WESTERN",Movie,6.9,2020.0,11433050.0,WESTERN,#001bff
740,0.0,"Animation, Adventure, Comedy, Drama, Family, M...",Spirit Riding Free: Ride Along Adventure,"ANIMATION,ADVENTURE,COMEDY,DRAMA,FAMILY,MYSTER...",Movie,5.9,2020.0,0.0,WESTERN,#ff3351
